# 没ネタ: 走れメロスっぽいなにか

In [ ]:
!pip install markovify mecab-python3

In [ ]:
import functools
import io
import re
import urllib.request
import zipfile


@functools.lru_cache(maxsize=None)
def download_content(url):
    """Download the content from the given URL."""
    return urllib.request.urlopen(url).read()


def download_text(identifier):
    """Download a text from the Aozorabunko Github repository.

    Example
    -------
    >>> print(download_text('35/1567'))
    """

    # Check IDs.
    m = re.fullmatch(r'(\d+)/(\d+)', identifier)
    if not m:
        raise ValueError(f'unrecognized identifier: {identifier}')
    author_id = int(m.group(1))
    work_id = int(m.group(2))

    # Download the card file.
    card = download_content(
            'https://raw.githubusercontent.com/aozorabunko/aozorabunko/master/cards/'
            f'{author_id:0=6}/card{work_id}.html'
        ).decode('utf-8')
    
    # Find the Zip file name.
    m = re.search(r'\d+_ruby_\d+\.zip', card)
    if not m:
        raise ValueError(f'card not found: {identifier}')
    zipname = m.group(0)

    # Download the Zip file.
    zipdata = zipfile.ZipFile(io.BytesIO(download_content(
            'https://raw.githubusercontent.com/aozorabunko/aozorabunko/master/cards/'
            f'{author_id:0=6}/files/{zipname}'
        )))

    # Extract the text from the Zip file.
    filename = zipdata.namelist()[0]
    text = zipdata.read(filename).decode('shift-jis')

    # Remove some anotations.
    text = re.split(r'\-{5,}', text)[2]
    text = re.split('底本：', text)[0]
    text = re.sub('｜', '', text)
    text = re.sub('《.+?》', '', text)
    text = re.sub('［＃.+?］', '', text)
    text = text.strip()

    return text

In [ ]:
text = download_text('35/1567')

In [ ]:
text

In [ ]:
import MeCab

parsed_text = MeCab.Tagger('-Owakati').parse(text)

In [ ]:
parsed_text

In [ ]:
import re
import markovify

normalized_text = parsed_text
normalized_text = re.sub('[、「」　]', '', normalized_text)
normalized_text = normalized_text.replace('！', '。')

model = markovify.Text(normalized_text.split('。'))


def make_sentence():
    s = model.make_sentence()
    if s is not None:
        return s.replace(' ', '') + '。'

In [ ]:
for i in range(10):
    print(make_sentence())